# Assembled Notebook — SK Agents (Derived)
Generated: 2025-11-08 06:13:44

In [ ]:

# %% [SETUP]
!pip install -U semantic-kernel
!pip -q uninstall -y pydrive2
!pip install -q azure-identity


In [ ]:

# %% [SETUP-ENV]
import os
os.environ["AZURE_OPENAI_ENDPOINT"]    = os.getenv("AZURE_OPENAI_ENDPOINT", "https://YOUR-AOAI.openai.azure.com")
os.environ["AZURE_OPENAI_API_KEY"]     = os.getenv("AZURE_OPENAI_API_KEY", "YOUR-AOAI-API-KEY")
os.environ["AZURE_OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-21")
os.environ["AZURE_OPENAI_DEPLOYMENT"]  = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-35-turbo")
print("AZURE_OPENAI_ENDPOINT:", os.environ["AZURE_OPENAI_ENDPOINT"])


In [ ]:

# %% [KERNEL]
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = Kernel()
service = AzureChatCompletion(
    service_id="azure",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)
kernel.add_service(service)
print("Kernel ready (Azure OpenAI)")


## Paste Mermaid → Generate Agents & Tools
Paste into `MERMAID` below and run.

In [ ]:

# %% [GENERATOR]
import re, keyword, textwrap

MERMAID = """
# Paste your Mermaid text here
# flowchart LR
#   APP[App Service]
#   SEARCH[Azure AI Search]
#   APP --> SEARCH
"""

def _slug(name: str) -> str:
    s = re.sub(r'[^a-zA-Z0-9_]+', '_', name.strip().lower())
    if s and s[0].isdigit():
        s = f"a_{s}"
    if keyword.iskeyword(s):
        s = s + "_x"
    return s.strip('_')

def extract_nodes(mermaid: str):
    boxes = re.findall(r'\[([^\[\]\n]+)\]', mermaid)
    rounds = re.findall(r'\(([^\(\)\n]+)\)', mermaid)
    return list(dict.fromkeys([t.strip() for t in (boxes + rounds) if t.strip()]))

def suggest_tools_from_name(name: str):
    name_l = name.lower()
    tools = []
    if any(k in name_l for k in ["search", "retriever", "rag"]):
        tools.append("tool_ai_search")
    if any(k in name_l for k in ["http", "api", "apim", "gateway"]):
        tools.append("tool_http")
    if any(k in name_l for k in ["storage", "blob", "files"]):
        tools.append("tool_storage")
    if any(k in name_l for k in ["calc", "math"]):
        tools.append("tool_calc")
    if any(k in name_l for k in ["key vault", "secret", "kv"]):
        tools.append("tool_key_vault")
    if any(k in name_l for k in ["openai", "llm", "model"]):
        tools.append("tool_model")
    return list(dict.fromkeys(tools))

# ---- Tool stubs ----
def tool_ai_search(query: str = "", **kwargs):
    return {"tool":"ai_search","query":query,"items":["doc1","doc2"],"kwargs":kwargs}

def tool_http(url: str, method: str = "GET", **kwargs):
    return {"tool":"http","url":url,"method":method,"status":200,"kwargs":kwargs}

def tool_storage(action: str = "put", path: str = "", **kwargs):
    return {"tool":"storage","action":action,"path":path,"ok":True,"kwargs":kwargs}

def tool_calc(expr: str, **kwargs):
    import ast, operator as op
    allowed = {ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul, ast.Div: op.truediv}
    def _eval(node):
        if isinstance(node, ast.Num): return node.n
        if isinstance(node, ast.BinOp) and type(node.op) in allowed:
            return allowed[type(node.op)](_eval(node.left), _eval(node.right))
        raise ValueError("Unsafe or unsupported expression")
    val = _eval(ast.parse(expr, mode='eval').body)
    return {"tool":"calc","expr":expr,"value":val}

def tool_key_vault(secret_name: str, **kwargs):
    return {"tool":"key_vault","name":secret_name,"value":"***MASKED***"}

def tool_model(prompt: str, **kwargs):
    try:
        import asyncio
        coro = kernel.invoke_prompt(prompt)
        result = asyncio.get_event_loop().run_until_complete(coro)
        return {"tool":"model","text":str(result)}
    except Exception as e:
        return {"tool":"model","error":str(e)}

TOOLS = {
    "tool_ai_search": tool_ai_search,
    "tool_http": tool_http,
    "tool_storage": tool_storage,
    "tool_calc": tool_calc,
    "tool_key_vault": tool_key_vault,
    "tool_model": tool_model,
}
print("Registered tools:", list(TOOLS.keys()))

# ---- Agent classes from nodes ----
nodes = extract_nodes(MERMAID)
agents_list = []
for name in nodes:
    slug = _slug(name)
    tools = suggest_tools_from_name(name)
    sysmsg = f"You are the '{name}' capability agent. Be concise; cite tools used."
    cls = f"""
class Agent_{slug}:
    def __init__(self, kernel):
        self.kernel = kernel
        self.name = "{name}"
        self.system_message = {sysmsg!r}
        self.skills = {tools!r}
    async def run(self, user_text: str) -> str:
        try:
            result = await self.kernel.invoke_prompt(self.system_message + "\n\nUser: " + user_text)
            return str(result)
        except Exception as e:
            return f"[{name} stub] SK invoke error: {{e}}"
    def available_tools(self):
        return [t for t in self.skills if t in TOOLS]
    def call(self, tool_name: str, **kwargs):
        fn = TOOLS.get(tool_name)
        if not fn: raise ValueError(f"Tool not found: {{tool_name}}")
        return fn(**kwargs)
agent_{slug} = Agent_{slug}(kernel)
"""
    exec(cls, globals())
    agents_list.append(f"agent_{slug}")

print("Agents:", agents_list if agents_list else "(none)")


In [ ]:

# %% [DEMO]
import asyncio

_agent_vars = [k for k,v in globals().items() if k.startswith("agent_")]
print("Agent instances:", _agent_vars)

async def demo():
    if not _agent_vars:
        return {"error":"no agents generated; paste MERMAID and run [GENERATOR] first"}
    agent = globals()[_agent_vars[0]]
    out = {"agent":agent.name, "tools":agent.available_tools()}
    try:
        out["llm"] = await agent.run("Say hello and state your purpose in one sentence.")
    except Exception as e:
        out["llm_error"] = str(e)
    if out["tools"]:
        t = out["tools"][0]
        try:
            out["tool_demo"] = agent.call(t, example="value")
        except Exception as e:
            out["tool_error"] = str(e)
    return out

try:
    loop = asyncio.get_running_loop()
    try:
        result = await demo()
    except SyntaxError:
        import nest_asyncio
        nest_asyncio.apply()
        result = loop.run_until_complete(demo())
except RuntimeError:
    result = asyncio.run(demo())

print("[DEMO RESULT]", result)
